<a href="https://colab.research.google.com/github/SeungPang11/Medicare-Drug-Spending-Data-Analysis/blob/main/Medicare_Drug_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drug Spending Data Analysis and Prediction**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
# downloaded = drive.CreateFile({'id': id})
# downloaded.GetContentFile('spending-2011-2015.csv')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
#sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
#spark = SparkSession.builder().master("local[1]").appName("SparkByExamples.com").getOrCreate()
drug = spark.read.csv("/content/drive/MyDrive/spending-2011-2015.csv", header=True, inferSchema=True)
drug.printSchema()

root
 |-- brand_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- claim_count: integer (nullable = true)
 |-- total_spending: double (nullable = true)
 |-- beneficiary_count: integer (nullable = true)
 |-- total_annual_spending_per_user: double (nullable = true)
 |-- unit_count: double (nullable = true)
 |-- average_cost_per_unit_weighted: double (nullable = true)
 |-- beneficiary_count_no_lis: integer (nullable = true)
 |-- average_beneficiary_cost_share_no_lis: double (nullable = true)
 |-- beneficiary_count_lis: integer (nullable = true)
 |-- average_beneficiary_cost_share_lis: double (nullable = true)
 |-- annual_change_in_average_cost_per_unit: double (nullable = true)



In [ ]:
drug2012 = spark.read.csv("/content/drive/MyDrive/spending-2012.csv", header=True, inferSchema=True)
drug2012.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- drugname_brand: string (nullable = true)
 |-- drugname_generic: string (nullable = true)
 |-- claim_count: integer (nullable = true)
 |-- total_spending: double (nullable = true)
 |-- user_count: double (nullable = true)
 |-- total_spending_per_user: double (nullable = true)
 |-- unit_count: double (nullable = true)
 |-- unit_cost_wavg: double (nullable = true)
 |-- user_count_non_lowincome: double (nullable = true)
 |-- out_of_pocket_avg_non_lowincome: double (nullable = true)
 |-- user_count_lowincome: double (nullable = true)
 |-- out_of_pocket_avg_lowincome: double (nullable = true)



In [ ]:
# drug.select('total_spending',format_string('%.1f',drug.total_spending.cast('float')).alias('total_spending_converted')).show()

In [ ]:
# Read Dataframes from 2011 to 2015
drug_df2011 = pd.read_csv("/content/drive/MyDrive/spending-2011.csv")
drug_df2012 = pd.read_csv("/content/drive/MyDrive/spending-2012.csv")
drug_df2013 = pd.read_csv("/content/drive/MyDrive/spending-2013.csv")
drug_df2014 = pd.read_csv("/content/drive/MyDrive/spending-2014.csv")
drug_df2015 = pd.read_csv("/content/drive/MyDrive/spending-2015.csv")

In [ ]:
drug_df2011.head(5)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,10 WASH,SULFACETAMIDE SODIUM,24,1569.19,16.00,98.07,5170.00,0.30,NaN,NaN,NaN,NaN
1,1,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",2472,57666.73,893.00,64.58,293160.00,0.20,422.00,42.35,471.00,7.55
2,2,60PSE-400GFN-20DM,GUAIFENESIN/DM/PSEUDOEPHEDRINE,12,350.10,11.00,31.83,497.00,0.70,NaN,NaN,NaN,NaN
3,3,8-MOP,METHOXSALEN,11,9003.26,NaN,NaN,298.00,30.21,NaN,NaN,NaN,NaN
4,4,A-B OTIC,ANTIPYRINE/BENZOCAINE,30,212.86,29.00,7.34,451.00,0.47,NaN,NaN,NaN,NaN


In [ ]:
drug_df2012.head(5)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",3486,75786.01,1261.00,60.10,405484.00,0.19,607.00,38.24,654.00,6.50
1,1,ABACAVIR,ABACAVIR SULFATE,20895,10337212.78,5618.00,1840.02,1330356.00,7.77,941.00,78.65,4677.00,0.72
2,2,ABELCET,AMPHOTERICIN B LIPID COMPLEX,465,585453.65,109.00,5371.13,59917.00,9.77,61.00,561.53,48.00,17.96
3,3,ABILIFY,ARIPIPRAZOLE,2572031,1758069618.10,376604.00,4668.22,82162175.54,21.67,66009.00,501.53,310595.00,21.23
4,4,ABILIFY DISCMELT,ARIPIPRAZOLE,4705,4529795.27,915.00,4950.60,192615.50,23.52,89.00,440.76,826.00,12.49


In [ ]:
drug_df2013.head(5)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",4413,95304.09,1633.00,58.36,522780.00,0.18,833.00,35.62,800.00,5.88
1,1,8-MOP,METHOXSALEN,14,19590.75,NaN,NaN,750.00,26.12,NaN,NaN,NaN,NaN
2,2,ABACAVIR,ABACAVIR SULFATE,59121,26923482.06,8360.00,3220.51,3761037.00,7.16,1594.00,282.05,6766.00,7.73
3,3,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,49,78230.03,49.00,1596.53,3473.00,22.53,14.00,44.84,35.00,0.23
4,4,ABELCET,AMPHOTERICIN B LIPID COMPLEX,435,582002.93,130.00,4476.95,56574.00,10.29,83.00,533.82,47.00,11.66


In [ ]:
drug_df2014.head(5)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",6788,150749.66,2769.00,54.44,813255.00,0.19,1481.00,32.14,1288.00,5.13
1,1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",1137,27282.97,619.00,44.08,137290.00,0.20,313.00,23.87,306.00,2.62
2,2,ABACAVIR,ABACAVIR SULFATE,68259,27961706.56,9076.00,3080.84,4341049.99,6.44,1870.00,263.72,7206.00,7.37
3,3,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,12201,17167220.41,1821.00,9427.36,766713.00,22.39,388.00,825.37,1433.00,11.77
4,4,ABELCET,AMPHOTERICIN B LIPID COMPLEX,419,514240.20,116.00,4433.11,55290.00,9.30,78.00,351.81,38.00,4.39


In [ ]:
drug_df2015.head(5)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",10749,257895.63,5173.00,49.85,1335132.00,0.19,2878.00,27.31,2295.00,3.99
1,1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",2943,73396.65,1470.00,49.93,369886.00,0.20,710.00,28.07,760.00,3.51
2,2,ABACAVIR,ABACAVIR SULFATE,69216,24578834.46,9319.00,2637.50,4424163.00,5.56,2046.00,236.94,7273.00,6.15
3,3,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,12216,16626213.80,1621.00,10256.76,759933.00,21.88,377.00,945.15,1244.00,24.19
4,4,ABELCET,AMPHOTERICIN B LIPID COMPLEX,439,552847.75,102.00,5420.08,47600.00,11.61,68.00,303.14,34.00,31.55


In [ ]:
# Some columns contain scientific notation (e) -> round 
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Check if numbers rounded 
drug_df2011.head(3)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,10 WASH,SULFACETAMIDE SODIUM,24,1569.19,16.00,98.07,5170.00,0.30,NaN,NaN,NaN,NaN
1,1,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",2472,57666.73,893.00,64.58,293160.00,0.20,422.00,42.35,471.00,7.55
2,2,60PSE-400GFN-20DM,GUAIFENESIN/DM/PSEUDOEPHEDRINE,12,350.10,11.00,31.83,497.00,0.70,NaN,NaN,NaN,NaN


In [ ]:
drug_df2012.head(3)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",3486,75786.01,1261.00,60.10,405484.00,0.19,607.00,38.24,654.00,6.50
1,1,ABACAVIR,ABACAVIR SULFATE,20895,10337212.78,5618.00,1840.02,1330356.00,7.77,941.00,78.65,4677.00,0.72
2,2,ABELCET,AMPHOTERICIN B LIPID COMPLEX,465,585453.65,109.00,5371.13,59917.00,9.77,61.00,561.53,48.00,17.96


In [ ]:
drug_df2013.head(3)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",4413,95304.09,1633.00,58.36,522780.00,0.18,833.00,35.62,800.00,5.88
1,1,8-MOP,METHOXSALEN,14,19590.75,NaN,NaN,750.00,26.12,NaN,NaN,NaN,NaN
2,2,ABACAVIR,ABACAVIR SULFATE,59121,26923482.06,8360.00,3220.51,3761037.00,7.16,1594.00,282.05,6766.00,7.73


In [ ]:
drug_df2014.head(3)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",6788,150749.66,2769.00,54.44,813255.00,0.19,1481.00,32.14,1288.00,5.13
1,1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",1137,27282.97,619.00,44.08,137290.00,0.20,313.00,23.87,306.00,2.62
2,2,ABACAVIR,ABACAVIR SULFATE,68259,27961706.56,9076.00,3080.84,4341049.99,6.44,1870.00,263.72,7206.00,7.37


In [ ]:
drug_df2015.head(3)

,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",10749,257895.63,5173.00,49.85,1335132.00,0.19,2878.00,27.31,2295.00,3.99
1,1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",2943,73396.65,1470.00,49.93,369886.00,0.20,710.00,28.07,760.00,3.51
2,2,ABACAVIR,ABACAVIR SULFATE,69216,24578834.46,9319.00,2637.50,4424163.00,5.56,2046.00,236.94,7273.00,6.15


In [ ]:
drug_df2011["drugname_brand"].unique()

array(['10 WASH', '1ST TIER UNIFINE PENTIPS', '60PSE-400GFN-20DM', ...,
       'ZYRTEC-D', 'ZYTIGA', 'ZYVOX'], dtype=object)

In [ ]:
drug_df2011["drugname_brand"].nunique()

3381

In [ ]:
drug_df2011["drugname_generic"].unique()

array(['SULFACETAMIDE SODIUM', 'PEN NEEDLE, DIABETIC',
       'GUAIFENESIN/DM/PSEUDOEPHEDRINE', ...,
       'CETIRIZINE HCL/PSEUDOEPHEDRINE', 'ABIRATERONE ACETATE',
       'LINEZOLID'], dtype=object)

In [ ]:
drug_df2011["drugname_generic"].nunique()

1804

In [ ]:
# Check for missing values 
# Columns with missing values 
# user_count, total_spending_per_user, user_count_non_lowincome, out_of_pocket_avg_non_lowincome, user_count_lowincome, out_of_pocket_avg_lowincome
drug_df2011.isnull().sum()

Unnamed: 0                           0
drugname_brand                       0
drugname_generic                     0
claim_count                          0
total_spending                       0
user_count                         206
total_spending_per_user            206
unit_count                           0
unit_cost_wavg                       0
user_count_non_lowincome           747
out_of_pocket_avg_non_lowincome    803
user_count_lowincome               747
out_of_pocket_avg_lowincome        789
dtype: int64

In [ ]:
drug_df2012.isnull().sum()

Unnamed: 0                           0
drugname_brand                       0
drugname_generic                     0
claim_count                          0
total_spending                       0
user_count                         193
total_spending_per_user            193
unit_count                           0
unit_cost_wavg                       0
user_count_non_lowincome           670
out_of_pocket_avg_non_lowincome    733
user_count_lowincome               670
out_of_pocket_avg_lowincome        717
dtype: int64

In [ ]:
drug_df2013.isnull().sum()

Unnamed: 0                           0
drugname_brand                       0
drugname_generic                     0
claim_count                          0
total_spending                       0
user_count                         159
total_spending_per_user            159
unit_count                           0
unit_cost_wavg                       0
user_count_non_lowincome           598
out_of_pocket_avg_non_lowincome    661
user_count_lowincome               598
out_of_pocket_avg_lowincome        645
dtype: int64

In [ ]:
drug_df2014.isnull().sum()

Unnamed: 0                           0
drugname_brand                       0
drugname_generic                     0
claim_count                          0
total_spending                       0
user_count                         150
total_spending_per_user            150
unit_count                           0
unit_cost_wavg                       0
user_count_non_lowincome           511
out_of_pocket_avg_non_lowincome    542
user_count_lowincome               511
out_of_pocket_avg_lowincome        519
dtype: int64

In [ ]:
drug_df2015.isnull().sum()

Unnamed: 0                           0
drugname_brand                       0
drugname_generic                     0
claim_count                          0
total_spending                       0
user_count                         169
total_spending_per_user            169
unit_count                           0
unit_cost_wavg                       0
user_count_non_lowincome           538
out_of_pocket_avg_non_lowincome    557
user_count_lowincome               538
out_of_pocket_avg_lowincome        540
dtype: int64

In [ ]:
# Check for dataframe shape - 3583 rows and 13 columns
drug_df2011.shape

(3583, 13)

In [ ]:
# Column names - 14 total
drug_df2011.columns

Index(['Unnamed: 0', 'drugname_brand', 'drugname_generic', 'claim_count',
       'total_spending', 'user_count', 'total_spending_per_user', 'unit_count',
       'unit_cost_wavg', 'user_count_non_lowincome',
       'out_of_pocket_avg_non_lowincome', 'user_count_lowincome',
       'out_of_pocket_avg_lowincome'],
      dtype='object')

In [ ]:
# Add year column to each yearly dataframes
drug_df2011.insert(0, "year", [2011] * drug_df2011.shape[0], allow_duplicates=True)
drug_df2012.insert(0, "year", [2012] * drug_df2012.shape[0], allow_duplicates=True)
drug_df2013.insert(0, "year", [2013] * drug_df2013.shape[0], allow_duplicates=True)
drug_df2014.insert(0, "year", [2014] * drug_df2014.shape[0], allow_duplicates=True)
drug_df2015.insert(0, "year", [2015] * drug_df2015.shape[0], allow_duplicates=True)

In [ ]:
drug_df2011.head(3)

,year,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,2011,0,10 WASH,SULFACETAMIDE SODIUM,24,1569.19,16.00,98.07,5170.00,0.30,NaN,NaN,NaN,NaN
1,2011,1,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",2472,57666.73,893.00,64.58,293160.00,0.20,422.00,42.35,471.00,7.55
2,2011,2,60PSE-400GFN-20DM,GUAIFENESIN/DM/PSEUDOEPHEDRINE,12,350.10,11.00,31.83,497.00,0.70,NaN,NaN,NaN,NaN


In [ ]:
drug_df2012.head(3)

,year,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,2012,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",3486,75786.01,1261.00,60.10,405484.00,0.19,607.00,38.24,654.00,6.50
1,2012,1,ABACAVIR,ABACAVIR SULFATE,20895,10337212.78,5618.00,1840.02,1330356.00,7.77,941.00,78.65,4677.00,0.72
2,2012,2,ABELCET,AMPHOTERICIN B LIPID COMPLEX,465,585453.65,109.00,5371.13,59917.00,9.77,61.00,561.53,48.00,17.96


In [ ]:
drug_df2013.head(3)

,year,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,2013,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",4413,95304.09,1633.00,58.36,522780.00,0.18,833.00,35.62,800.00,5.88
1,2013,1,8-MOP,METHOXSALEN,14,19590.75,NaN,NaN,750.00,26.12,NaN,NaN,NaN,NaN
2,2013,2,ABACAVIR,ABACAVIR SULFATE,59121,26923482.06,8360.00,3220.51,3761037.00,7.16,1594.00,282.05,6766.00,7.73


In [ ]:
drug_df2014.head(3)

,year,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,2014,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",6788,150749.66,2769.00,54.44,813255.00,0.19,1481.00,32.14,1288.00,5.13
1,2014,1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",1137,27282.97,619.00,44.08,137290.00,0.20,313.00,23.87,306.00,2.62
2,2014,2,ABACAVIR,ABACAVIR SULFATE,68259,27961706.56,9076.00,3080.84,4341049.99,6.44,1870.00,263.72,7206.00,7.37


In [ ]:
drug_df2015.head(3)

,year,Unnamed: 0,drugname_brand,drugname_generic,claim_count,total_spending,user_count,total_spending_per_user,unit_count,unit_cost_wavg,user_count_non_lowincome,out_of_pocket_avg_non_lowincome,user_count_lowincome,out_of_pocket_avg_lowincome
0,2015,0,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",10749,257895.63,5173.00,49.85,1335132.00,0.19,2878.00,27.31,2295.00,3.99
1,2015,1,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",2943,73396.65,1470.00,49.93,369886.00,0.20,710.00,28.07,760.00,3.51
2,2015,2,ABACAVIR,ABACAVIR SULFATE,69216,24578834.46,9319.00,2637.50,4424163.00,5.56,2046.00,236.94,7273.00,6.15


In [ ]:
# Concat all dataframes from 2011-2015
alldrug_df = pd.concat([drug_df2011, drug_df2012, drug_df2013, drug_df2014, drug_df2015], keys=drug_df2011.columns)
alldrug_df.head(5)

year  Unnamed: 0            drugname_brand  \
year 0  2011           0                   10 WASH   
     1  2011           1  1ST TIER UNIFINE PENTIPS   
     2  2011           2         60PSE-400GFN-20DM   
     3  2011           3                     8-MOP   
     4  2011           4                  A-B OTIC   

                      drugname_generic  claim_count  total_spending  \
year 0            SULFACETAMIDE SODIUM           24         1569.19   
     1            PEN NEEDLE, DIABETIC         2472        57666.73   
     2  GUAIFENESIN/DM/PSEUDOEPHEDRINE           12          350.10   
     3                     METHOXSALEN           11         9003.26   
     4           ANTIPYRINE/BENZOCAINE           30          212.86   

        user_count  total_spending_per_user  unit_count  unit_cost_wavg  \
year 0       16.00                    98.07     5170.00            0.30   
     1      893.00                    64.58   293160.00            0.20   
     2       11.00                    31.83      497.00            0.70   
     3         NaN                      NaN      298.00           30.21   
     4       29.00                     7.34      451.00            0.47   

        user_count_non_lowincome  out_of_pocket_avg_non_lowincome  \
year 0                       NaN                              NaN   
     1                    422.00                            42.35   
     2                       NaN                              NaN   
     3                       NaN                              NaN   
     4                       NaN                              NaN   

        user_count_lowincome  out_of_pocket_avg_lowincome  
year 0                   NaN                          NaN  
     1                471.00                         7.55  
     2                   NaN                          NaN  
     3                   NaN                          NaN  
     4                   NaN                          NaN

In [ ]:
# 17302 rows and 14 columns
alldrug_df.shape

(17302, 14)

In [ ]:
import pyspark.sql.functions as F
drug = drug.withColumn("Total_Spending", F.ceil("total_spending"))

In [ ]:
drug.printSchema()

root
 |-- brand_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- claim_count: integer (nullable = true)
 |-- total_spending: double (nullable = true)
 |-- beneficiary_count: integer (nullable = true)
 |-- total_annual_spending_per_user: double (nullable = true)
 |-- unit_count: double (nullable = true)
 |-- average_cost_per_unit_weighted: double (nullable = true)
 |-- beneficiary_count_no_lis: integer (nullable = true)
 |-- average_beneficiary_cost_share_no_lis: double (nullable = true)
 |-- beneficiary_count_lis: integer (nullable = true)
 |-- average_beneficiary_cost_share_lis: double (nullable = true)
 |-- annual_change_in_average_cost_per_unit: double (nullable = true)
 |-- Total_Spending: double (nullable = true)



In [ ]:
# Spark SQL
drug.registerTempTable("drug_sql")

/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [ ]:
drug.printSchema()

root
 |-- brand_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- claim_count: integer (nullable = true)
 |-- total_spending: double (nullable = true)
 |-- beneficiary_count: integer (nullable = true)
 |-- total_annual_spending_per_user: double (nullable = true)
 |-- unit_count: double (nullable = true)
 |-- average_cost_per_unit_weighted: double (nullable = true)
 |-- beneficiary_count_no_lis: integer (nullable = true)
 |-- average_beneficiary_cost_share_no_lis: double (nullable = true)
 |-- beneficiary_count_lis: integer (nullable = true)
 |-- average_beneficiary_cost_share_lis: double (nullable = true)
 |-- annual_change_in_average_cost_per_unit: double (nullable = true)
 |-- total_spending_converted: double (nullable = true)
 |-- Total_Drug_Spending: long (nullable = true)



In [ ]:
# Top Selling Brands 2011
spark.sql("SELECT year, brand_name, Total_Drug_Spending \
          FROM drug_sql \
          WHERE year = '2011' \
          ORDER BY Total_Drug_Spending DESC").show(10, False)

+----+----------------------+-------------------+
|year|brand_name            |Total_Drug_Spending|
+----+----------------------+-------------------+
|2011|plavix                |3656740021         |
|2011|lipitor               |2672901532         |
|2011|seroquel              |2045551133         |
|2011|nexium                |1970138398         |
|2011|advair diskus         |1664865681         |
|2011|zyprexa               |1625350331         |
|2011|abilify               |1469661103         |
|2011|lantus/lantus solostar|1467024269         |
|2011|crestor               |1416315013         |
|2011|actos                 |1294152166         |
+----+----------------------+-------------------+
only showing top 10 rows



In [ ]:
# Top Selling Brands 2012
spark.sql("SELECT year, brand_name, Total_Drug_Spending \
          FROM drug_sql \
          WHERE year = '2012' \
          ORDER BY Total_Drug_Spending DESC").show(10, False)

+----+----------------------+-------------------+
|year|brand_name            |Total_Drug_Spending|
+----+----------------------+-------------------+
|2012|nexium                |2124303797         |
|2012|lantus/lantus solostar|1945151505         |
|2012|advair diskus         |1886312551         |
|2012|crestor               |1789825372         |
|2012|abilify               |1758069619         |
|2012|plavix                |1690680325         |
|2012|spiriva               |1603177831         |
|2012|cymbalta              |1453826873         |
|2012|atorvastatin calcium  |1347833772         |
|2012|namenda               |1327416590         |
+----+----------------------+-------------------+
only showing top 10 rows



In [ ]:
# Top Selling Brands 2013
spark.sql("SELECT year, brand_name, Total_Drug_Spending \
          FROM drug_sql \
          WHERE year = '2013' \
          ORDER BY Total_Drug_Spending DESC").show(10, False)

+----+----------------------+-------------------+
|year|brand_name            |Total_Drug_Spending|
+----+----------------------+-------------------+
|2013|lantus/lantus solostar|2683090322         |
|2013|nexium                |2526306069         |
|2013|advair diskus         |2262995221         |
|2013|crestor               |2216193162         |
|2013|abilify               |2107091536         |
|2013|cymbalta              |1961132000         |
|2013|spiriva               |1958581193         |
|2013|namenda               |1564775426         |
|2013|januvia               |1460522984         |
|2013|revlimid              |1349964593         |
+----+----------------------+-------------------+
only showing top 10 rows



In [ ]:
# Top Selling Brands 2014
spark.sql("SELECT year, brand_name, Total_Drug_Spending \
          FROM drug_sql \
          WHERE year = '2014' \
          ORDER BY Total_Drug_Spending DESC").show(10, False)

+----+----------------------+-------------------+
|year|brand_name            |Total_Drug_Spending|
+----+----------------------+-------------------+
|2014|lantus/lantus solostar|3742568386         |
|2014|sovaldi               |3106960981         |
|2014|nexium                |2660421778         |
|2014|crestor               |2543786426         |
|2014|abilify               |2527319032         |
|2014|advair diskus         |2276374749         |
|2014|spiriva               |2158439228         |
|2014|januvia               |1775295533         |
|2014|revlimid              |1671622657         |
|2014|duloxetine hcl        |1426877759         |
+----+----------------------+-------------------+
only showing top 10 rows



In [ ]:
# Top Selling Brands 2015
spark.sql("SELECT year, brand_name, Total_Drug_Spending \
          FROM drug_sql \
          WHERE year = '2015' \
          ORDER BY Total_Drug_Spending DESC").show(10, False)

+----+----------------------+-------------------+
|year|brand_name            |Total_Drug_Spending|
+----+----------------------+-------------------+
|2015|harvoni               |7030633486         |
|2015|lantus/lantus solostar|4359504167         |
|2015|crestor               |2883122485         |
|2015|advair diskus         |2270015726         |
|2015|spiriva               |2191465784         |
|2015|januvia               |2131952446         |
|2015|revlimid              |2077425469         |
|2015|nexium                |2012920920         |
|2015|lyrica                |1766473720         |
|2015|humira/humira pen     |1662291828         |
+----+----------------------+-------------------+
only showing top 10 rows



In [1]:
# Find the top selling brands by year from 2011-2015; Total_Drug_Spending
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

print("Best Selling Brands by Year")
windowSpec  = Window.partitionBy("year").orderBy(col("Total_Drug_Spending").desc())
window_df = drug.withColumn("row_number",row_number().over(windowSpec)) \
    .filter(col("row_number") == 1) \
    .select(col("year"), col("brand_name"), col("Total_Drug_Spending"))\
    .show(truncate=False)

ModuleNotFoundError: ignored

In [ ]:
# Find the top selling brands from 2011-2015; Sum of total_spending
spark.sql("SELECT brand_name, SUM(total_spending_converted) as Total_Spending \
          FROM drug_sql \
          WHERE Total_Spending >= 10000\
          GROUP BY brand_name \
          ORDER BY Total_Spending DESC").show(20, False)

+----------------------+--------------------+
|brand_name            |Total_Spending      |
+----------------------+--------------------+
|lantus/lantus solostar|1.41973386467E10    |
|nexium                |1.12940909595E10    |
|crestor               |1.08492424562E10    |
|advair diskus         |1.03605639267E10    |
|abilify               |9.4345696308E9      |
|spiriva               |9.2000929931E9      |
|harvoni               |7.73052605807E9     |
|januvia               |7.27144037304E9     |
|revlimid              |6.92531058483E9     |
|namenda               |6.03927284026E9     |
|lyrica                |5.63923480437E9     |
|plavix                |5.4591748942300005E9|
|copaxone              |5.34323239011E9     |
|humira/humira pen     |5.04517741165E9     |
|enbrel                |4.850244236719999E9 |
|cymbalta              |4.61741864976E9     |
|oxycontin             |4.56921562081E9     |
|sovaldi               |4.43955922732E9     |
|zetia                 |4.34506995

In [ ]:
spark.sql("SELECT SUM(total_spending) \
          FROM drug_sql \
          WHERE brand_name = 'procalamine'").show()

In [ ]:
spark.sql("SELECT year, brand_name, total_spending \
          FROM drug_sql \
          WHERE total_spending >= 1000").show()